In [25]:
import os
import datetime as dt
import pandas as pd

In [26]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"
labelled_data_fnames = [f for f in os.listdir(labelled_data_dir)]
master_data_location  = f"{repo_dir}/data/labelled_master_data_{dt.date.today()}.csv"
print(labelled_data_fnames)
print(master_data_location)

['yj2369_labelling_assignment_2022-04-13.csv', 'yj2369_labelling_assignment_2022-04-28.csv', 'gm2858_labelling_assignment_2022-04-21.csv', 'gm2858_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-21.csv', 'ac4119_labelling_assignment_2022-04-09.csv', 'ac4119_labelling_assignment_2022-04-21.csv', 'gm2858_labelling_assignment_2022-04-28.csv', 'yp2201_labelling_assignment_2022-04-28.csv', 'ac4119_labelling_assignment_2022-04-13.csv', 'ac4119_labelling_assignment_2022-04-28.csv', 'yp2201_labelling_assignment_2022-04-13.csv', 'gm2858_labelling_assignment_2022-04-13.csv', 'yj2369_labelling_assignment_2022-04-09.csv', 'yj2369_labelling_assignment_2022-04-21.csv']
/Users/ameliachu/repos/nlu-reddit-toxicity-dataset/data/labelled_master_data_2022-05-12.csv


In [27]:
labels = ['toxicity', 'severe_toxicity', 'identity_attack', 'insult', 'profanity', 'threat']
selected_columns = ['example_id', 'preceding_comment', 'comment_for_evaluation',
       'following_comment'] + labels 
list_of_labelled_data = []
for fname in labelled_data_fnames:
    
    single = pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] 
    single['date'] = fname.split('_')[-1].split('.')[0]
    
    list_of_labelled_data.append(single)


In [28]:
labelled_master_data = pd.concat(list_of_labelled_data).drop_duplicates().sort_values(by='date').reset_index(drop=True)

In [29]:
labelled_master_data.to_csv(master_data_location,index=False)

In [30]:
labelled_master_data.head(5)

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,date
0,7398,LOOK AT THOSE JOBLESS CLAIMS,I use fidelity.,what a gift having that political news shock t...,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-09
1,262301,GME just making ITM calls cheaper by making th...,SNDL looking good,PLTR always a bitch!,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-09
2,26260,"Keep holding retards we close about $150, we o...",Ask nicely & it will bounce uppways 😜,Fucktards are going to crash the market...,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-09
3,263278,vxx falling with red candles on spy\n\nsomethi...,Thinking JPow comes out tomorrow on presser dr...,FAANG,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-09
4,266897,Because when the price can go up 15-20 points ...,"It's okay, BB, you can go whenever you feel li...",Love it lmao. I’ve gotten like -100 in 30 seco...,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-09


In [31]:
len(labelled_master_data)

3403

In [32]:
labelled_master_data = labelled_master_data.drop_duplicates(subset='example_id',keep='first')

In [33]:
len(labelled_master_data )

2441

In [34]:
labelled_master_data['has_toxicity'] = labelled_master_data[labels].sum(axis=1)

In [36]:
len(labelled_master_data[labelled_master_data['has_toxicity'] >0])

483

In [38]:
master_data_location  = f"{repo_dir}/data/all_labelled_data_{dt.date.today()}.csv"
labelled_master_data.to_csv(master_data_location,index=False)

In [14]:
toxic_ids = list(labelled_master_data[labelled_master_data['has_toxicity'] > 0]['example_id'].values)

In [17]:
results_dir = f"{repo_dir}/artifacts/results"
human_2_pd = pd.read_csv(f"{results_dir}/baseline_human-2_2022-05-07.csv") 

included_in_baseline = list(human_2_pd['example_id'].values)

In [18]:
identity_mentions = [270830, 146593]
other_interesting = [91067, 146593, 379993]

In [19]:
must_hold_ids = list(set(toxic_ids + included_in_baseline + identity_mentions + other_interesting))

In [20]:
n_nontoxic_ids = 800 - len(must_hold_ids)

In [39]:
n_nontoxic_ids

293

In [21]:
remaining_sample = labelled_master_data[labelled_master_data['has_toxicity'] == 0]
remaining_sample = remaining_sample[~remaining_sample['example_id'].isin(must_hold_ids)].sample(n_nontoxic_ids )

In [22]:
must_hold_sample = labelled_master_data[labelled_master_data['example_id'].isin(must_hold_ids)]
final_master_table = pd.concat([must_hold_sample, remaining_sample])[selected_columns]

In [23]:
final_master_table

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat
6,275362,Which one of u bought my stocks? >:(,posted this in the other thread but the more i...,That thing is 1.64% rn. JPow has to say somet...,0.0,0.0,0.0,0.0,1.0,0.0
12,30108,Alexa play down by 311.,No one cares about 10 year bonds bro - thats j...,Sorry you were born the bad kind of retarded.,0.0,0.0,0.0,0.0,1.0,0.0
21,316461,Today = no Vaseline,The entire market is fucking hemorrhaging,Imagine putting your money in bonds lol,0.0,0.0,0.0,0.0,1.0,0.0
22,316957,definitely. keep buying guys !!!!! HURRRDURRr,damn so my puts are fukd,"Quick, everyone buy Into the bull trap again!",0.0,0.0,0.0,0.0,1.0,0.0
23,318752,Big if true.,Do you routinely comment your opinion on topic...,"They get all 4, fuck that. \n\nJust one Warloc...",1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1778,219798,Buy calls. Plenty of opportunity,"Here, take a downvote!","you can also see it on yahoo, go to the fullsc...",0.0,0.0,0.0,0.0,0.0,0.0
1420,68599,It’s a dark day lmaoooooo,How do I block someone,UWMC let’s go guys 🚀🚀🚀,0.0,0.0,0.0,0.0,0.0,0.0
2313,31882,My amc 15 calls ain’t lookin too good,market starting to bounce a little... get that...,The market is completely dumping and is the in...,0.0,0.0,0.0,0.0,0.0,0.0
1341,482069,Thank you to whoever sold me the PLTR $26 FD c...,yeah i seem to have timed that one almost perf...,Watching Rat Race. One of best comedies of all...,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
master_data_location  = f"{repo_dir}/data/labelled_master_data_{dt.date.today()}.csv"
final_master_table.to_csv(master_data_location,index=False)